<a href="https://colab.research.google.com/github/Yang-yamper/practice-for-graduation/blob/main/%E3%83%95%E3%83%AC%E3%83%BC%E3%83%A0%E6%8A%BD%E5%87%BA%EF%BC%86%E6%B0%B4%E5%A2%97%E3%81%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


札をとってる瞬間をフレーム数で抜き出してフレームを抽出

In [6]:
def save_frame_range(video_path, start_frame, stop_frame, step_frame,
                     dir_path, basename, ext='jpg'):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return

    os.makedirs(dir_path, exist_ok=True)
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    for n in range(start_frame, stop_frame, step_frame):
        cap.set(cv2.CAP_PROP_POS_FRAMES, n)
        ret, frame = cap.read()
        if ret:
            cv2.imwrite('{}_{}.{}'.format(base_path, str(n).zfill(digit), ext), frame)
        else:
            return

save_frame_range("/content/drive/MyDrive/IMG_7675.mov",
                 240, 300, 1,
                 '/content/drive/MyDrive/result_range/sample1', 'sample_video_img')

札撮ってる瞬間を秒数で指定して画像抽出

In [23]:
def save_frame_range_sec(video_path, start_sec, stop_sec, step_sec,
                         dir_path, basename, ext='jpg'):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return

    os.makedirs(dir_path, exist_ok=True)
    base_path = os.path.join(dir_path, basename)

    digit = len(str(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))))

    fps = cap.get(cv2.CAP_PROP_FPS)
    fps_inv = 1 / fps

    sec = start_sec
    while sec < stop_sec:
        n = round(fps * sec)
        cap.set(cv2.CAP_PROP_POS_FRAMES, n)
        ret, frame = cap.read()
        if ret:
            cv2.imwrite(
                '{}_{}_{:.2f}.{}'.format(
                    base_path, str(n).zfill(digit), n * fps_inv, ext
                ),
                frame
            )
        else:
            return
        sec += step_sec

save_frame_range_sec("/content/drive/MyDrive/IMG_7642.mov",
                     4, 6, 0.1,
                     '/content/drive/MyDrive/result_range/right', 'sample_video_img')

画像の水増し

In [30]:
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2


DATA_DIR = '/content/drive/MyDrive/result_range/left'  # データディレクトリ
IMAGE_NAME = '/content/drive/MyDrive/result_range/left/sample_video_img_243_8.89.jpg'  # 対象画像ファイル
SAVE_DIR = os.path.join(DATA_DIR, '/content/drive/MyDrive/result_range/left')  # 生成画像の保存先ディレクトリ

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img_array = cv2.imread(os.path.join(DATA_DIR, IMAGE_NAME),)  # 画像読み込み
img_array = img_array.reshape((1,) + img_array.shape)  # 4次元データに変換（flow()に渡すため）

# 保存先ディレクトリが存在しない場合、作成する。
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

# flow()により、ランダム変換したイメージのバッチを作成。
# 指定したディレクトリに生成画像を保存する。
i = 0
for batch in datagen.flow(img_array, batch_size=1,
                          save_to_dir=SAVE_DIR, save_prefix='caruta', save_format='jpg'):
    i += 1
    if i == 50:
        break  # 停止しないと無限ループ